In [4]:
import os
import shutil
import random

# Define the directory containing your images
data_dir = 'Augmented1'

# Define the directory where you want to store the train and test datasets
train_dir = 'train_dir'
test_dir = 'test_dir'

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define the subcategories (classes)
subcategories = ['Africanized Honey Bees (Killer Bees)_augmented', 'Aphids_augmented', 'Armyworms_augmented']

# Define the train-test split ratio (e.g., 80% train, 20% test)
train_ratio = 0.8

# Iterate over each subcategory
for subcategory in subcategories:
    # Get the list of images for the current subcategory
    images = os.listdir(os.path.join(data_dir, subcategory))
    # Shuffle the list of images
    random.shuffle(images)
    # Calculate the number of images for the train set based on the train_ratio
    num_train = int(len(images) * train_ratio)
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    # Create subdirectories in train and test directories
    os.makedirs(os.path.join(train_dir, subcategory), exist_ok=True)
    os.makedirs(os.path.join(test_dir, subcategory), exist_ok=True)
    # Copy images to train directory
    for image in train_images:
        src = os.path.join(data_dir, subcategory, image)
        dst = os.path.join(train_dir, subcategory, image)
        shutil.copy(src, dst)
    # Copy images to test directory
    for image in test_images:
        src = os.path.join(data_dir, subcategory, image)
        dst = os.path.join(test_dir, subcategory, image)
        shutil.copy(src, dst)


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
# Define directories for train and validation datasets
train_dir = 'train_dir'
test_dir = 'test_dir'

# Define data generators for train and validation datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set the image size and batch size
img_size = (256, 256)
batch_size = 32

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 445 images belonging to 3 classes.
Found 112 images belonging to 3 classes.
Epoch 1/10
14/14 [==============================] - 27s 2s/step - loss: 3.5053 - accuracy: 0.3461 - val_loss: 1.0780 - val_accuracy: 0.3482
Epoch 2/10
14/14 [==============================] - 24s 2s/step - loss: 1.0818 - accuracy: 0.3506 - val_loss: 1.1767 - val_accuracy: 0.3482
Epoch 3/10
14/14 [==============================] - 25s 2s/step - loss: 1.0711 - accuracy: 0.3775 - val_loss: 1.0275 - val_accuracy: 0.4107
Epoch 4/10
14/14 [==============================] - 25s 2s/step - loss: 1.0608 - accuracy: 0.4404 - val_loss: 1.0224 - val_accuracy: 0.4554
Epoch 5/10
14/14 [==============================] - 25s 2s/step - loss: 1.0592 - accuracy: 0.4382 - val_loss: 1.0189 - val_accuracy: 0.4821
Epoch 6/10
14/14 [==============================] - 26s 2s/step - loss: 1.0535 - accuracy: 0.4607 - val_loss: 1.0804 - val_accuracy: 0.4643
Epoch 7/10
14/14 [==============================] - 27s 2s/step - loss: 1.0996

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# Load pre-trained ResNet50 model without including top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model layers
base_model.trainable = False

# Add your custom classifier on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(lr=0.0001),  # Reduced learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more epochs
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


c:\Users\YASH\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
17/17 [==============================] - 76s 4s/step - loss: 1.6304 - accuracy: 0.3265 - val_loss: 1.1285 - val_accuracy: 0.4138
Epoch 2/20
17/17 [==============================] - 78s 5s/step - loss: 1.3490 - accuracy: 0.3638 - val_loss: 1.0815 - val_accuracy: 0.4236
Epoch 3/20
17/17 [==============================] - 94s 6s/step - loss: 1.2597 - accuracy: 0.3507 - val_loss: 1.0743 - val_accuracy: 0.3990
Epoch 4/20
17/17 [==============================] - 80s 5s/step - loss: 1.2582 - accuracy: 0.3321 - val_loss: 1.0680 - val_accuracy: 0.4039
Epoch 5/20
17/17 [==============================] - 72s 4s/step - loss: 1.1952 - accuracy: 0.3955 - val_loss: 1.0691 - val_accuracy: 0.3941
Epoch 6/20
17/17 [==============================] - 73s 4s/step - loss: 1.1844 - accuracy: 0.3750 - val_loss: 1.0660 - val_accuracy: 0.4039
Epoch 7/20
17/17 [==============================] - 70s 4s/step - loss: 1.2161 - accuracy: 0.3396 - val_loss: 1.0544 - val_accuracy: 0.4729
Epoch 8/20
17/17 [==

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models

# Define directories for train and validation datasets
train_dir = 'train_dir'
test_dir = 'test_dir'

# Define data generators for train and validation datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set the image size and batch size
img_size = (256, 256)
batch_size = 32

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG19 model without the top layers (include_top=False)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers so they are not trained during fine-tuning
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 445 images belonging to 3 classes.
Found 112 images belonging to 3 classes.
80150528/80134624 [==============================] - 16s 0us/step
Epoch 1/10
14/14 [==============================] - 165s 12s/step - loss: 4.4900 - accuracy: 0.3798 - val_loss: 0.8536 - val_accuracy: 0.6786
Epoch 2/10
14/14 [==============================] - 161s 12s/step - loss: 1.3198 - accuracy: 0.4944 - val_loss: 0.7657 - val_accuracy: 0.6607
Epoch 3/10
14/14 [==============================] - 176s 13s/step - loss: 0.8734 - accuracy: 0.6112 - val_loss: 0.6574 - val_accuracy: 0.7679
Epoch 4/10
14/14 [==============================] - 177s 13s/step - loss: 0.7976 - accuracy: 0.6607 - val_loss: 0.6432 - val_accuracy: 0.7679
Epoch 5/10
14/14 [==============================] - 160s 12s/step - loss: 0.7913 - accuracy: 0.6360 - val_loss: 0.6012 - val_accuracy: 0.7589
Epoch 6/10
14/14 [==============================] - 167s 12s/step - loss: 0.7274 - accuracy: 0.6944 - val_loss: 0.5446 - val_accuracy: 0.7857
